In [15]:
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

In [16]:
# Definición del modelo

class AlexNet(nn.Module):
  def __init__(self, num_classes = 1000):
    super(AlexNet, self).__init__()
    self.characteristic = nn.Sequential(
        # Bloque morado
        nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
        nn.ReLU(inplace = True),
        nn.BatchNorm1d(96),
        # Bloque azul 1
        nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
        # Bloque azul 2
        nn.Conv2d(64, 256, kernel_size=5, stride=1, padding=2),
        nn.ReLU(inplace = True),
        nn.BatchNorm2d(256),
        # Bloque azul 3
        nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
        # Bloque azul 4
        nn.Conv2d(256, 384, kernel_size=5, stride=1, padding=2),
        nn.ReLU(inplace = True),
        # Bloque azul 6
        nn.Conv2d(384, 384, kernel_size=5, stride=1, padding=2),
        nn.ReLU(inplace = True),
        # Bloque azul 7
        nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace = True),
        # Bloque azul 8
        nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
    )

    self.flatter = nn.AdaptiveAvgPool2d((6, 6))

    self.classificator_nn = nn.Sequential(
        nn.Dropout(),
        nn.Linear(6 * 6 * 256, 4096),
        nn.ReLU(inplace = True),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace = True),
        nn.Linear(4096, num_classes),
        nn.Softmax(dim = 1)
    )

  def forward(self, x_data):
    value_tracker = self.characteristic(x_data)
    value_tracker = self.flatter(value_tracker)
    value_tracker = self.classificator_nn(value_tracker)
    return value_tracker

'''
Nota #1: num_classes es el número de cosas que quiero detectar
Nota #2: El profesor suguiere usar 64 en lugar de 96 para que no tarde tanto
Nota #3: El BatchNorm1d es un normalizador, se sugiere comentarlo
Nota #4: Esto BatchNorm1d es una absoluta, esto es una raiz cuadrada BatchNorm2d
Nota #5: 6 * 6 * 256 define las entradas de la NNFC
Nota #6: Los 4096 son parte del modelo
'''

'\nNota #1: num_classes es el número de cosas que quiero detectar\nNota #2: El profesor suguiere usar 64 en lugar de 96 para que no tarde tanto\nNota #3: El BatchNorm1d es un normalizador, se sugiere comentarlo\nNota #4: Esto BatchNorm1d es una absoluta, esto es una raiz cuadrada BatchNorm2d\nNota #5: 6 * 6 * 256 define las entradas de la NNFC\nNota #6: Los 4096 son parte del modelo\n'

In [17]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
])

# Función para registrar las activaciones de las capas intermedias
activations = {}
def get_activation(name):
  def hook(model, input, output):
    activations[name] = output.detach()
  return hook

alexnet = AlexNet()

# Registrar ganchos (hooks) en capas intermedias
print(alexnet.characteristic[3].register_forward_hook(get_activation('conv1')))
print(alexnet.characteristic[6].register_forward_hook(get_activation('conv2')))